In [2]:
import tensorflow as tf
import numpy as np

import os

In [4]:
# create training and validation data
batch_size = 64
main_path = 'D:/keras_datasets/aclImdb'
train_path = os.path.join(main_path, 'train')
test_path = os.path.join(main_path, 'test')

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_path,
    batch_size = batch_size,
    validation_split = .2,
    subset = 'training',
    seed = 1337   # always set a seed for reproducibility
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_path,
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',
    seed = 1337 # always set a seed for reproducibility
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_path, batch_size = batch_size
)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.


ValueError: If `subset` is set, `validation_split` must be set, and inversely.